## 2. <a id='toc2_'></a>[Problem 2: Labor adjustment costs](#toc0_)

In [28]:
#impports and magics
import numpy as np
from scipy import linalg
from scipy import optimize
import sympy as sm

# local modules
import opg2py
from opg2py import ProfitClass
model = ProfitClass()

ImportError: cannot import name 'ProfitClass' from 'opg2py' (/Users/natacha/Desktop/projects-2023-ctrl-alt-elite/examproject/opg2py.py)

You own a hair salon. You employ hairdressers, $\ell_t$, to produce haircuts, $y_t = \ell_t$.

The wage for each haridresser is $w$.

The demand for haircuts implies that the price of haircuts you can charge is $p_t = \kappa_t y_t^{-\eta}$, where $\kappa_t$ is a demand-shock and $\eta \in (0,1)$ measures the elasticity of demand.

Profits are:

$$
\Pi_t = p_t y_t - w \ell_t = \kappa_t \ell_t^{1-\eta} - w \ell_t
$$

Baseline parameters are:
- $\eta = 0.5$
- $w = 1.0$

**Question 1:** Verify numerically that $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ maximises profits, for $\kappa\in\left\{1.0 , 2.0\right\}$.

As statet in the problemset the profits are given by:
$$
\Pi_t = p_t y_t - w \ell_t = \kappa_t \ell_t^{1-\eta} - w \ell_t
$$
This can be simplified for the following for $\kappa=1$
$$
\Pi_t=\ell_t^{0.5}-\ell_t
$$
And for $\kappa=2$
$$
\Pi_t=2\ell_t^{0.5}-\ell_t
$$

We see that when $\kappa=1$ we have the following value for $\ell_t$:

$$
\ell_t=\left(\frac{(1-0.5)*1}{1}\right)^{\frac{1}{0.5}}
\\
\ell_t=\left(\frac{0.5}{1}\right)^{\frac{1}{0.5}}=0.25
$$

We see that when $\kappa=2$ we have the following value for $\ell_t$:
$$
\ell_t=\left(\frac{(1-0.5)*2}{1}\right)^{\frac{1}{0.5}}
\\
\ell_t=\left(\frac{1}{1}\right)^{\frac{1}{0.5}}
\\
\ell_t=1^2=1
$$

We solve this numiricly by using sympy


In [26]:
model = ProfitClass(do_print=False)
model.solve_numerical_kappa1()

NameError: name 'ProfitClass' is not defined

In [29]:
model = ProfitClass(do_print=False)
model.solve_numerical_kappa2()

NameError: name 'ProfitClass' is not defined

In [24]:
#define values for the problem
ell_t, kappa_t, eta, w = sm.symbols('ell_t, kappa_t, eta, w')

#define the profit function
Pi = kappa_t*ell_t**(1-eta) -w*ell_t

# Set the values of eta and w
eta_val = 0.5
w_val = 1.0

# Choose a specific value for kappa
kappa_val = 1.0

# Calculate the derivative of the profit equation with respect to ell_t
d_Pi = sm.diff(Pi, ell_t)

# Find the value of ell_t that maximizes profits
optimal_ell = sm.solve(d_Pi, ell_t)[0]

# Substitute the parameter values into the optimal_ell expression
optimal_ell_val = optimal_ell.subs([(kappa_t, kappa_val), (eta, eta_val), (w, w_val)])

optimal_ell_val

0.250000000000000

In [25]:
# Choose a specific value for kappa
kappa_val = 2.0

# Calculate the derivative of the profit equation with respect to ell_t
d_Pi = sm.diff(Pi, ell_t)

# Find the value of ell_t that maximizes profits
optimal_ell = sm.solve(d_Pi, ell_t)[0]

# Substitute the parameter values into the optimal_ell expression
optimal_ell_val = optimal_ell.subs([(kappa_t, kappa_val), (eta, eta_val), (w, w_val)])

optimal_ell_val

1.00000000000000

In [12]:
kappa = 1.0
eta = 0.5
w = 1.0

# Calculate profits for different values of l_t
l_values = np.linspace(0.1, 10.0, 1000)
profits = kappa * l_values**(1 - eta) - w * l_values

# Find the value of ell_t that maximizes profits
max_profit_index = np.argmax(profits)
max_profit_l = l_values[max_profit_index]

max_profit_l


0.24864864864864866

In [11]:
kappa = 2.0

# Calculate profits for different values of l_t
profits = kappa * l_values**(1 - eta) - w * l_values

# Find the value of l_t that maximizes profits
max_profit_index = np.argmax(profits)
max_profit_ell = l_values[max_profit_index]

max_profit_ell

1.001801801801802

We now consider a *dynamic* version of the model.

* The demand-shock is a so-called AR(1) in logs, 

$$
\log \kappa_{t} = \rho \log \kappa_{t-1} + \epsilon_{t},\,\,\, \epsilon_{t+1} \sim \mathcal{N}(-0.5\sigma_{\epsilon}^2,\sigma_{\epsilon})
$$

* Any hiring or firing implies a fixed adjustment cost, $\iota > 0 $.
* Future profits are discounted with a monthly factor of $R \in (0,1)$.

The initial demand shock is $\kappa_{-1} = 1$ and the planning horizon is 10 years, i.e. 120 months so $t \in \{0,1,2,\dots,119\}$. Initially you don't have any employees, $\ell_{-1}=0$

The *ex post* value of the salon is *conditional* on the shock series is:

$$
h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119}) = \left[\sum_{t=0}^{119}R^{-t}\left[\kappa_{t}\ell_{t}^{1-\eta}-w\ell_{t}-\boldsymbol{1}_{\ell_{t}\neq\ell_{t-1}}\iota\right]\right]
$$

The *ex ante* expected value of the salon can be approximated by

$$
H = \mathbb{E}[h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119})] \approx \frac{1}{K}\sum_{k=0}^{K} h(\epsilon_0^k,\epsilon_1^k,\dots,\epsilon_{119}^k)
$$

where each $k\in\{0,1,\dots,K-1\}$ is a random shock series. Maximizing profitability means maximizing $H$.


Baseline parameters are: 

- $\rho = 0.90$
- $\iota = 0.01$
- $\sigma_{\epsilon} = 0.10$
- $R = \left(1+0.01\right)^{1/12}$

**Question 2:** Calculate $H$ if the policy  $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ from question 1 is followed. Choose $K$ so the approximation is good enough to not affect your results substantially.

In [30]:

# Set baseline parameters
eta = 0.5
w = 1.0
rho = 0.90
iota = 0.01
sigma_epsilon = 0.10
R = (1 + 0.01) ** (1 / 12)
T = 120  # Planning horizon

# Initialize variables
epsilon = np.random.normal(-0.5 * sigma_epsilon ** 2, sigma_epsilon, size=T)
kappa = np.zeros(T)
kappa[0] = 1.0
ell = np.zeros(T)
ell[0] = ((1 - eta) * kappa[0] / w) ** (1 / eta)
value_function = np.zeros(T)

# Simulate the shock series and compute the value function
for t in range(1, T):
    kappa[t] = np.exp(rho * np.log(kappa[t - 1]) + epsilon[t])
    ell[t] = ((1 - eta) * kappa[t] / w) ** (1 / eta)
    value_function[t] = R ** -t * (kappa[t] * ell[t] ** (1 - eta) - w * ell[t] - (ell[t] != ell[t - 1]) * iota)

# Calculate the expected value of the salon
K = 10000  # Number of shock series
expected_value = np.mean([np.sum(value_function) for _ in range(K)])

print("Expected value of the salon (H):", expected_value)

Expected value of the salon (H): 27.133002030766722


Next, we consider policies on the form:

$$

\ell_{t}=\begin{cases}
\ell_t^{\ast}  & \text{if }\left|\ell_{t-1}-\ell_t^{\ast} \right|>\Delta\\
\ell_{t-1} & \text{else }
\end{cases}
\\
\text{where}\,\,\ell_t^{\ast} = \left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}} \\

$$
With $\Delta \geq 0$ and $\Delta = 0$ being the previous policy.


**Question 3:** Calculate $H$ if the policy above was followed with $\Delta = 0.05$. Does it improve profitability?

In [ ]:
skriv svar


**Question 4:** Find the optimal $\Delta$ maximizing $H$. Illustrate your result.

In [ ]:
skriv svar her


**Question 5:** Suggest an alternative policy you believe might improve profitability. Implement and test your policy.